# Minesweeper LLM Competition - Custom GRPO Training

## Goal
Finetune an LLM with LoRA using GRPO to play Minesweeper by:
- **Input**: JSON game state (board configuration)
- **Output**: JSON action (reveal or flag a cell)

Teams will compete to train the best Minesweeper-playing LLM!

## Training Approach
- **Model**: Qwen/Qwen3-4B (from /root/.cache/huggingface/hub)
- **Method**: GRPO (Group Relative Policy Optimization)
- **Framework**: Unsloth (2-6x faster, 70% less VRAM)
- **Hardware**: AMD GPU (ROCm)
- **Thinking Mode**: Disabled (non-thinking mode for pure JSON output)

# Load Model with Unsloth

Load Qwen3-4B with LoRA configuration:

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # Qwen3-4B supports up to 32K natively; 4096 is plenty for Minesweeper
lora_rank = 32         # LoRA rank — 32 gives more reasoning capacity than 16

# Load model with Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen3-4B",
    load_in_4bit = False,
    max_seq_length = max_seq_length,
    torch_dtype = torch.bfloat16,
)

print(f"Model: Qwen/Qwen3-4B")
print(f"Model device: {model.device}")
print(f"Max seq length: {max_seq_length}")
print(f"LoRA rank: {lora_rank}")
print("Model loaded successfully!")

# Add LoRA Adapters

Add LoRA layers for efficient finetuning:

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,           # alpha = rank → scaling factor = 1.0 (stable training)
    lora_dropout = 0.05,              # Small dropout for regularization
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)
print(f"LoRA config: rank={lora_rank}, alpha={lora_rank}, dropout=0.05")
model.print_trainable_parameters()

# Minesweeper Game Implementation

Custom Minesweeper environment supporting:
- Customizable board size and mine count
- Actions: reveal or flag cells
- Win: reveal all safe cells
- Lose: reveal a mine

In [ ]:
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Set
import random

@dataclass
class MinesweeperGame:
    rows: int
    cols: int
    num_mines: int
    seed: Optional[int] = None
    _rng: random.Random = field(init=False, repr=False)
    _board: List[List[int]] = field(init=False, repr=False)  # -1 = mine, 0-8 = count
    _revealed: Set[Tuple[int, int]] = field(init=False, repr=False, default_factory=set)
    _flagged: Set[Tuple[int, int]] = field(init=False, repr=False, default_factory=set)
    _state: str = field(default="ongoing", init=False, repr=False)

    def __post_init__(self):
        if self.num_mines >= self.rows * self.cols:
            raise ValueError("Too many mines for board size")
        self._rng = random.Random(self.seed)
        self._board = [[0 for _ in range(self.cols)] for _ in range(self.rows)]
        self._place_mines()
        self._calculate_numbers()

    def _place_mines(self):
        """Place mines randomly on the board"""
        positions = [(r, c) for r in range(self.rows) for c in range(self.cols)]
        mine_positions = self._rng.sample(positions, self.num_mines)
        for r, c in mine_positions:
            self._board[r][c] = -1

    def _calculate_numbers(self):
        """Calculate numbers for each cell based on adjacent mines"""
        for r in range(self.rows):
            for c in range(self.cols):
                if self._board[r][c] == -1:
                    continue
                count = 0
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        if dr == 0 and dc == 0:
                            continue
                        nr, nc = r + dr, c + dc
                        if 0 <= nr < self.rows and 0 <= nc < self.cols:
                            if self._board[nr][nc] == -1:
                                count += 1
                self._board[r][c] = count

    def _reveal_cell(self, row: int, col: int) -> bool:
        """Reveal a cell. Returns True if valid move, False if invalid.
        Uses iterative flood-fill to avoid recursion limit on large boards.
        (Issue #11: was recursive; Issue typo: fixed 'bself' -> 'self')
        """
        if not (0 <= row < self.rows and 0 <= col < self.cols):
            return False
        if (row, col) in self._revealed or (row, col) in self._flagged:
            return False

        stack = [(row, col)]
        while stack:
            r, c = stack.pop()
            if (r, c) in self._revealed:
                continue

            self._revealed.add((r, c))

            # Hit a mine!
            if self._board[r][c] == -1:
                self._state = "failed"
                return True

            # Auto-reveal neighbors if cell is 0
            if self._board[r][c] == 0:
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        if dr == 0 and dc == 0:
                            continue
                        nr, nc = r + dr, c + dc
                        if (0 <= nr < self.rows and 0 <= nc < self.cols
                                and (nr, nc) not in self._revealed
                                and (nr, nc) not in self._flagged):
                            stack.append((nr, nc))

        return True

    def _flag_cell(self, row: int, col: int) -> bool:
        """Flag/unflag a cell. Returns True if valid, False if invalid"""
        if not (0 <= row < self.rows and 0 <= col < self.cols):
            return False
        if (row, col) in self._revealed:
            return False
        
        if (row, col) in self._flagged:
            self._flagged.remove((row, col))
        else:
            self._flagged.add((row, col))
        return True

    def do_action(self, action: dict) -> str:
        """Execute an action and return a status string.

        Returns one of:
          'ok'               - valid move executed
          'mine'             - revealed a mine (game over)
          'win'              - game won after this move
          'invalid_format'   - bad action dict / missing keys / bad types
          'out_of_bounds'    - coordinates outside the board
          'already_revealed' - cell was already revealed
          'flagged_cell'     - tried to reveal a flagged cell
          'invalid_flag'     - tried to flag a revealed cell
          'game_over'        - game was already over before this call

        (Issue #13: previously set state='failed' for ALL invalid moves,
         conflating formatting errors with hitting a mine.)
        """
        if self._state != "ongoing":
            return "game_over"

        if not isinstance(action, dict):
            self._state = "failed"
            return "invalid_format"

        action_type = action.get("type")
        row = action.get("row")
        col = action.get("col")

        if action_type not in ["reveal", "flag"] or row is None or col is None:
            self._state = "failed"
            return "invalid_format"

        try:
            row, col = int(row), int(col)
        except (ValueError, TypeError):
            self._state = "failed"
            return "invalid_format"

        if not (0 <= row < self.rows and 0 <= col < self.cols):
            self._state = "failed"
            return "out_of_bounds"

        if action_type == "reveal":
            if (row, col) in self._revealed:
                self._state = "failed"
                return "already_revealed"
            if (row, col) in self._flagged:
                self._state = "failed"
                return "flagged_cell"
            valid = self._reveal_cell(row, col)
        else:
            if (row, col) in self._revealed:
                self._state = "failed"
                return "invalid_flag"
            valid = self._flag_cell(row, col)

        if not valid:
            self._state = "failed"
            return "invalid_format"

        self._check_win()

        if self._state == "failed":
            return "mine"
        if self._state == "success":
            return "win"
        return "ok"

    def _check_win(self):
        """Check if player has won"""
        total_cells = self.rows * self.cols
        safe_cells = total_cells - self.num_mines
        if len(self._revealed) == safe_cells:
            self._state = "success"

    def get_visible_board(self) -> List[List[str]]:
        """Get board state as player sees it"""
        visible = []
        for r in range(self.rows):
            row = []
            for c in range(self.cols):
                if (r, c) in self._flagged:
                    row.append('F')
                elif (r, c) in self._revealed:
                    val = self._board[r][c]
                    row.append('*' if val == -1 else str(val))
                else:
                    row.append('.')
            visible.append(row)
        return visible

    def state(self) -> str:
        return self._state

    def pretty_print(self) -> str:
        """Pretty print the board"""
        visible = self.get_visible_board()
        lines = []
        
        # Header
        header = "   " + " ".join(f"{i:2d}" for i in range(self.cols))
        lines.append(header)
        lines.append("  " + "─" * (self.cols * 3 + 1))
        
        # Board
        for r, row in enumerate(visible):
            line = f"{r:2d}│ " + "  ".join(row)
            lines.append(line)
        
        return "\n".join(lines)

# JSON Input/Output Format

## Input Format (Game State)
```json
{
  "board": [
    ["1", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", "."]
  ],
  "rows": 6,
  "cols": 6,
  "mines": 5,
  "flags_placed": 0,
  "cells_revealed": 0
}
```

## Output Format (Action)
```json
{"type": "reveal", "row": 2, "col": 3}
```
or
```json
{"type": "flag", "row": 1, "col": 4}
```

In [ ]:
import json
import re

# ──────────────────────────────────────────────────────────────────────
# Minesweeper Logic Helpers — used by prompt AND reward functions
# ──────────────────────────────────────────────────────────────────────

def _compute_safe_cells(game: MinesweeperGame) -> list:
    """Find cells that are logically guaranteed safe.
    A cell is safe if any adjacent revealed number already has all its
    mines accounted for by flags (remaining_mines == 0)."""
    safe = set()
    for r in range(game.rows):
        for c in range(game.cols):
            if (r, c) not in game._revealed:
                continue
            val = game._board[r][c]
            if val <= 0:
                continue
            flags = 0
            unrevealed = []
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    if dr == 0 and dc == 0:
                        continue
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < game.rows and 0 <= nc < game.cols:
                        if (nr, nc) in game._flagged:
                            flags += 1
                        elif (nr, nc) not in game._revealed:
                            unrevealed.append((nr, nc))
            if val - flags == 0:
                for cell in unrevealed:
                    safe.add(cell)
    return [list(c) for c in safe]


def _compute_mine_cells(game: MinesweeperGame) -> list:
    """Find cells that are logically guaranteed mines.
    A cell is a mine if an adjacent number has remaining_mines ==
    remaining unrevealed neighbors."""
    mines = set()
    for r in range(game.rows):
        for c in range(game.cols):
            if (r, c) not in game._revealed:
                continue
            val = game._board[r][c]
            if val <= 0:
                continue
            flags = 0
            unrevealed = []
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    if dr == 0 and dc == 0:
                        continue
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < game.rows and 0 <= nc < game.cols:
                        if (nr, nc) in game._flagged:
                            flags += 1
                        elif (nr, nc) not in game._revealed:
                            unrevealed.append((nr, nc))
            remaining = val - flags
            if remaining > 0 and remaining == len(unrevealed):
                for cell in unrevealed:
                    mines.add(cell)
    return [list(c) for c in mines]


def _is_logically_safe(game: MinesweeperGame, row: int, col: int) -> bool:
    """Check if (row, col) appears in the set of logically safe cells."""
    return [row, col] in _compute_safe_cells(game)


def _is_logically_mine(game: MinesweeperGame, row: int, col: int) -> bool:
    """Check if (row, col) appears in the set of logically certain mines."""
    return [row, col] in _compute_mine_cells(game)


# ──────────────────────────────────────────────────────────────────────
# Prompt formatting — concise, hint-enriched, JSON-only output
# ──────────────────────────────────────────────────────────────────────

def format_state_for_llm(game: MinesweeperGame) -> str:
    """Convert game state to an optimized prompt for LLM.

    Key design choices (backed by research):
    - Provide pre-computed logical hints so the model doesn't need full
      constraint-satisfaction from scratch.
    - Keep instructions terse → less chance of exceeding max_completion_length.
    - Explicitly forbid explanation text → pure JSON output.
    - Include remaining_mines count to guide flagging strategy.
    """
    board = game.get_visible_board()
    state = {
        "board": board,
        "rows": game.rows,
        "cols": game.cols,
        "mines": game.num_mines,
        "flags_placed": len(game._flagged),
        "cells_revealed": len(game._revealed),
        "remaining_mines": game.num_mines - len(game._flagged),
    }

    # Pre-compute logical deductions as hints
    safe_cells = _compute_safe_cells(game)
    mine_cells = _compute_mine_cells(game)

    hint_lines = []
    if safe_cells:
        hint_lines.append(f"Logically SAFE cells (reveal one): {safe_cells[:6]}")
    if mine_cells:
        hint_lines.append(f"Logically CERTAIN mines (flag one): {mine_cells[:6]}")
    if not safe_cells and not mine_cells:
        hint_lines.append("No cells can be logically deduced — pick the least risky unrevealed cell.")

    hint_section = "\n".join(hint_lines)

    prompt = f"""You are an expert Minesweeper solver. Output ONE action as JSON only.

RULES:
- Numbers show how many of their 8 neighbors are mines.
- Subtract flagged neighbors from the number to find remaining mines.
- If remaining mines == remaining unrevealed neighbors → all are mines → FLAG.
- If remaining mines == 0 → all unrevealed neighbors are safe → REVEAL.
- Never reveal a flagged cell or flag a revealed cell.
- Prefer logically deducible moves over guessing.

{json.dumps(state, indent=2)}

"."=unrevealed "F"=flagged "0"-"8"=adjacent mine count

{hint_section}

Respond ONLY with a JSON object:
{{"type":"reveal"|"flag","row":<int>,"col":<int>}}"""

    return prompt


def parse_llm_action(response: str) -> dict:
    """Extract JSON action from LLM response.

    Finds all JSON-like objects and returns the LAST one matching the
    expected schema. LLMs typically place their final answer at the end.
    """
    best = None
    for match in re.finditer(r'\{[^{}]*\}', response):
        try:
            action = json.loads(match.group())
            if ("type" in action and "row" in action and "col" in action
                    and action["type"] in ["reveal", "flag"]):
                best = action
        except json.JSONDecodeError:
            continue
    return best

# ── Quick test ──
game = MinesweeperGame(rows=6, cols=6, num_mines=5)
prompt = format_state_for_llm(game)
print(prompt)
print(f"\n--- Prompt length: {len(prompt)} chars ---")

# Test Model Before Training

See how the base model performs without finetuning:

In [ ]:
from transformers import TextStreamer

game = MinesweeperGame(rows=6, cols=6, num_mines=5, seed=42)
prompt = format_state_for_llm(game)

# Qwen3: disable thinking mode → no <think>...</think> block, pure JSON output
text = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False,   # Qwen3: non-thinking mode for concise JSON
)

print("=== Base Model Response ===")
output = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0.7,       # Qwen3 non-thinking recommended: temp=0.7
    top_p = 0.8,             # Qwen3 non-thinking recommended: top_p=0.8
    top_k = 20,              # Qwen3 non-thinking recommended: top_k=20
    max_new_tokens = 128,
    do_sample = True,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

# GRPO Reward Functions

Define reward functions to guide the model's learning:

In [ ]:
import numpy as np

# ──────────────────────────────────────────────────────────────────────
# Reward 1: Valid JSON format
# ──────────────────────────────────────────────────────────────────────

def valid_json_reward(completions, **kwargs):
    """Reward valid JSON action format. Also rewards conciseness."""
    scores = []
    for completion in completions:
        response = completion[0]["content"].strip()
        action = parse_llm_action(response)

        if action is None:
            scores.append(-5.0)  # Invalid format
            continue

        # Bonus for pure JSON (no extra text)
        try:
            parsed = json.loads(response)
            if "type" in parsed and "row" in parsed and "col" in parsed:
                scores.append(3.0)  # Perfect — pure JSON only
                continue
        except json.JSONDecodeError:
            pass

        # Valid JSON but with extra surrounding text
        json_match = re.search(r'\{[^{}]*\}', response)
        extra_chars = len(response) - len(json_match.group()) if json_match else len(response)
        if extra_chars < 10:
            scores.append(2.0)
        elif extra_chars < 50:
            scores.append(1.0)
        elif extra_chars < 200:
            scores.append(-0.5)
        else:
            scores.append(-2.0)  # Way too verbose

    return scores


# ──────────────────────────────────────────────────────────────────────
# Reward 2: Gameplay — all 12 scoring criteria implemented
# ──────────────────────────────────────────────────────────────────────

def gameplay_scores(completions, **kwargs):
    """
    Complete gameplay reward implementing all 12 scoring criteria:

    1.  Flag cell that IS a mine        → +15
    2.  Flag cell that is NOT a mine    → -10
    3.  Reveal cell that IS a mine      → -25
    4.  Reveal cell that is safe        → +10 (guess) / +15 (logically deducible)
    5.  Flag already flagged cell       → -12
    6.  Reveal already revealed cell    → -12
    7.  Out of bounds                   → -15
    8.  Total flags > total mines       → -10 (additional)
    9.  Invalid JSON                    → -50
    10. Win the game                    → +100
    11. Reveal a flagged cell           → -8
    12. Flag a revealed cell            → -8
    """
    scores = []

    seeds = kwargs.get("seed", [])
    move_histories = kwargs.get("move_history", [])

    for idx, completion in enumerate(completions):
        response = completion[0]["content"]
        action = parse_llm_action(response)

        # ── Criterion 9: Invalid JSON ──
        if action is None:
            scores.append(-50.0)
            continue

        # ── Reconstruct game state ──
        if idx >= len(seeds) or idx >= len(move_histories):
            scores.append(0.0)
            continue

        seed = seeds[idx]
        move_history_raw = move_histories[idx]
        if isinstance(move_history_raw, str):
            move_history = json.loads(move_history_raw)
        else:
            move_history = move_history_raw

        game = MinesweeperGame(rows=6, cols=6, num_mines=5, seed=seed)
        for prev_action in move_history:
            game.do_action(prev_action)

        row, col = action["row"], action["col"]
        action_type = action["type"]

        # ── Criterion 7: Out of bounds ──
        if not (0 <= row < game.rows and 0 <= col < game.cols):
            scores.append(-15.0)
            continue

        score = 0.0

        if action_type == "reveal":
            # ── Criterion 6: Reveal already revealed cell ──
            if (row, col) in game._revealed:
                scores.append(-12.0)
                continue

            # ── Criterion 11: Reveal a flagged cell ──
            if (row, col) in game._flagged:
                scores.append(-8.0)
                continue

            # ── Criterion 3: Reveal a mine ──
            if game._board[row][col] == -1:
                scores.append(-25.0)
                continue

            # ── Criterion 4: Reveal safe cell ──
            if _is_logically_safe(game, row, col):
                score += 15.0   # Logically deduced safe cell
            else:
                score += 10.0   # Random / guessed safe cell

            # Small bonus for revealing cells adjacent to numbers (information-rich)
            board = game.get_visible_board()
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    nr, nc = row + dr, col + dc
                    if 0 <= nr < game.rows and 0 <= nc < game.cols:
                        if board[nr][nc] in ['1', '2', '3', '4', '5', '6', '7', '8']:
                            score += 1.0
                            break
                else:
                    continue
                break

            # ── Criterion 10: Check for win after this reveal ──
            game_copy = MinesweeperGame(rows=6, cols=6, num_mines=5, seed=seed)
            for prev_action in move_history:
                game_copy.do_action(prev_action)
            game_copy.do_action(action)
            if game_copy.state() == "success":
                score += 100.0

        elif action_type == "flag":
            # ── Criterion 5: Flag already flagged cell ──
            if (row, col) in game._flagged:
                scores.append(-12.0)
                continue

            # ── Criterion 12: Flag a revealed cell ──
            if (row, col) in game._revealed:
                scores.append(-8.0)
                continue

            # ── Criterion 1: Flag a mine (correct) ──
            if game._board[row][col] == -1:
                if _is_logically_mine(game, row, col):
                    score += 20.0   # Logically deduced mine
                else:
                    score += 15.0   # Correct but guessed

            # ── Criterion 2: Flag a non-mine (wrong) ──
            else:
                score += -10.0

            # ── Criterion 8: Total flags > total mines ──
            new_flag_count = len(game._flagged) + 1
            if new_flag_count > game.num_mines:
                score -= 10.0

            # ── Criterion 10: Check for win after this flag ──
            game_copy = MinesweeperGame(rows=6, cols=6, num_mines=5, seed=seed)
            for prev_action in move_history:
                game_copy.do_action(prev_action)
            game_copy.do_action(action)
            if game_copy.state() == "success":
                score += 100.0

        else:
            scores.append(-10.0)
            continue

        scores.append(score)

    return scores


# ──────────────────────────────────────────────────────────────────────
# Reward 3: Strategic play — rewards logical deduction over guessing
# ──────────────────────────────────────────────────────────────────────

def strategic_reward(completions, **kwargs):
    """Reward strategic play patterns:
    - Choosing logically deducible moves when available
    - Opening in corners/edges (lower mine probability on fresh boards)
    - Penalize ignoring available deductions
    """
    scores = []
    seeds = kwargs.get("seed", [])
    move_histories = kwargs.get("move_history", [])

    for idx, completion in enumerate(completions):
        response = completion[0]["content"]
        action = parse_llm_action(response)

        if action is None:
            scores.append(0.0)
            continue

        if idx >= len(seeds) or idx >= len(move_histories):
            scores.append(0.0)
            continue

        seed = seeds[idx]
        mh_raw = move_histories[idx]
        move_history = json.loads(mh_raw) if isinstance(mh_raw, str) else mh_raw

        game = MinesweeperGame(rows=6, cols=6, num_mines=5, seed=seed)
        for prev in move_history:
            game.do_action(prev)

        row, col = action["row"], action["col"]
        action_type = action["type"]
        score = 0.0

        if not (0 <= row < game.rows and 0 <= col < game.cols):
            scores.append(0.0)
            continue

        # ── Fresh game opening strategy ──
        if len(game._revealed) == 0 and action_type == "reveal":
            corners = [(0, 0), (0, game.cols - 1),
                       (game.rows - 1, 0), (game.rows - 1, game.cols - 1)]
            if (row, col) in corners:
                score += 2.0   # Corners have only 3 neighbors → safer

        # ── Reward choosing logically deducible moves ──
        safe_cells = _compute_safe_cells(game)
        mine_cells = _compute_mine_cells(game)

        if action_type == "reveal" and [row, col] in safe_cells:
            score += 5.0   # Chose a provably safe cell
        elif action_type == "flag" and [row, col] in mine_cells:
            score += 5.0   # Chose a provably mine cell
        elif safe_cells or mine_cells:
            # Deducible moves existed but agent didn't pick one
            score -= 3.0

        scores.append(score)

    return scores


print("✅ All reward functions defined:")
print("   1. valid_json_reward   — format + conciseness")
print("   2. gameplay_scores     — all 12 criteria")
print("   3. strategic_reward    — logical deduction bonuses")

# Create Training Dataset

Generate diverse game states for training:

In [ ]:
from datasets import Dataset

def generate_game_states(num_samples=2000, rows=6, cols=6, num_mines=5,
                         rng_seed=42):
    """
    Generate diverse Minesweeper game states with CURRICULUM LEARNING.

    Distribution by game phase (backed by Bengio et al. 2009):
    - 15% fresh games (0 moves)   → learn opening strategy
    - 25% early game  (1-2 moves) → learn basic deduction
    - 40% mid game    (3-8 moves) → learn complex constraint satisfaction
    - 20% late game   (9+ moves)  → learn endgame / flagging

    Also includes flag-in-progress states (10%) so the model
    learns when/how to flag.

    IMPORTANTLY: Stores seed + move_history (as JSON string) so reward
    function can reconstruct the EXACT game state.
    """
    np.random.seed(rng_seed)
    random.seed(rng_seed)

    dataset_items = []
    attempts = 0
    max_attempts = num_samples * 5

    # Move-count distribution for curriculum
    move_bins = [
        (0, 0, 0.15),    # Fresh
        (1, 2, 0.25),    # Early
        (3, 8, 0.40),    # Mid
        (9, 20, 0.20),   # Late
    ]

    while len(dataset_items) < num_samples and attempts < max_attempts:
        attempts += 1

        # Sample which phase
        phase_rand = np.random.random()
        cumulative = 0
        min_moves, max_moves_range = 0, 0
        for mn, mx, prob in move_bins:
            cumulative += prob
            if phase_rand < cumulative:
                min_moves, max_moves_range = mn, mx
                break
        num_moves = np.random.randint(min_moves, max_moves_range + 1)

        seed = np.random.randint(100000)
        game = MinesweeperGame(rows=rows, cols=cols, num_mines=num_mines, seed=seed)
        move_history = []

        # Occasionally include flag actions in history (10% chance per move)
        for _ in range(num_moves):
            board = game.get_visible_board()
            unrevealed = [(r, c) for r in range(rows) for c in range(cols)
                         if board[r][c] == '.']

            if not unrevealed or game.state() != "ongoing":
                break

            # 10% chance to flag instead of reveal (to train flag awareness)
            if np.random.random() < 0.10 and len(game._flagged) < num_mines:
                r, c = random.choice(unrevealed)
                action = {"type": "flag", "row": r, "col": c}
            else:
                r, c = random.choice(unrevealed)
                action = {"type": "reveal", "row": r, "col": c}

            game.do_action(action)
            move_history.append(action)

        # Only add ongoing games
        if game.state() == "ongoing":
            prompt_text = format_state_for_llm(game)
            dataset_items.append({
                "prompt": [{"role": "user", "content": prompt_text}],
                "seed": seed,
                "move_history": json.dumps(move_history),
            })

    dataset_items = dataset_items[:num_samples]
    return Dataset.from_list(dataset_items)

# Generate training dataset
print("Generating training dataset with curriculum learning...")
dataset = generate_game_states(num_samples=2000, rows=6, cols=6, num_mines=5)
print(f"Created {len(dataset)} training examples (all ongoing games)")

# Distribution analysis
fresh_count = sum(1 for item in dataset if item["move_history"] == "[]")
move_counts = [len(json.loads(item["move_history"])) for item in dataset]
print(f"\n  Fresh games (0 moves): {fresh_count} ({fresh_count/len(dataset)*100:.1f}%)")
print(f"  Early game (1-2):      {sum(1 for m in move_counts if 1 <= m <= 2)} ({sum(1 for m in move_counts if 1 <= m <= 2)/len(dataset)*100:.1f}%)")
print(f"  Mid game (3-8):        {sum(1 for m in move_counts if 3 <= m <= 8)} ({sum(1 for m in move_counts if 3 <= m <= 8)/len(dataset)*100:.1f}%)")
print(f"  Late game (9+):        {sum(1 for m in move_counts if m >= 9)} ({sum(1 for m in move_counts if m >= 9)/len(dataset)*100:.1f}%)")
print(f"  Avg moves per state:   {np.mean(move_counts):.1f}")

# Show example
print("\nExample training prompt (first 300 chars):")
print(dataset[0]["prompt"][0]["content"][:300] + "...")

# Configure GRPO Training

Set up GRPO trainer with all hyperparameters:

In [ ]:
from trl import GRPOConfig, GRPOTrainer

# ── Lengths ──
max_prompt_length = 700    # Increased: prompts now include logic hints
max_completion_length = 200  # Short JSON output — no reasoning text

# ── GRPO Configuration (research-backed) ──
# Sources: DeepSeekMath paper, TRL docs, Open-R1 blog, DAPO paper
training_args = GRPOConfig(
    # === Generation ===
    temperature = 0.9,           # Slightly below 1.0; Qwen3 non-thinking recommends 0.7
    top_p = 0.95,                # Qwen3 best practice
    top_k = 20,                  # Qwen3 best practice

    # === Qwen3: Disable thinking mode during GRPO generation ===
    chat_template_kwargs = {"enable_thinking": False},

    # === Optimization ===
    learning_rate = 2e-5,        # Lower LR → more stable RL training
    weight_decay = 0.01,
    warmup_ratio = 0.05,         # Shorter warmup for RL
    lr_scheduler_type = "cosine",  # Cosine > linear (Loshchilov & Hutter 2017)
    optim = "adamw_8bit",        # 8-bit Adam saves VRAM (Open-R1 lesson 5)
    max_grad_norm = 0.5,         # Tighter gradient clipping for stability

    # === Batch sizes ===
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    num_generations = 8,         # More generations → better reward estimation

    # === Lengths ===
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,

    # === Training duration ===
    max_steps = 500,             # Adjust based on compute budget
    save_steps = 100,

    # === GRPO specific ===
    # beta=0.0 (default) — KL term excluded per Open-Reasoner-Zero findings
    # loss_type="dapo" (default) — eliminates length bias (DAPO paper)
    # scale_rewards="group" (default) — normalize within group
    num_iterations = 1,          # Standard single-iteration GRPO

    # === Reward weighting (gameplay >> format >> strategy) ===
    reward_weights = [0.15, 0.70, 0.15],

    # === Output ===
    report_to = "none",
    output_dir = "minesweeper_grpo_v2",
    seed = 42,
    bf16 = True,
)

print("Training configuration:")
print(f"  Model:               Qwen/Qwen3-4B (non-thinking mode)")
print(f"  Max steps:           {training_args.max_steps}")
print(f"  Generations/state:   {training_args.num_generations}")
print(f"  Learning rate:       {training_args.learning_rate}")
print(f"  LR scheduler:       {training_args.lr_scheduler_type}")
print(f"  Max grad norm:       {training_args.max_grad_norm}")
print(f"  Loss type:           {training_args.loss_type}")
print(f"  Beta (KL penalty):   {training_args.beta}")
print(f"  Reward weights:      {training_args.reward_weights}")
print(f"  Prompt/Completion:   {max_prompt_length}/{max_completion_length}")
print(f"  Temperature:         {training_args.temperature}")
print(f"  Top-p / Top-k:       {training_args.top_p} / {training_args.top_k}")
print(f"  Thinking mode:       DISABLED (via chat_template_kwargs)")
print(f"  LoRA rank:           {lora_rank}")

In [ ]:
from transformers import TrainerCallback

class MinesweeperEvalCallback(TrainerCallback):
    """Periodically play games during training and log win rate + metrics."""

    def __init__(self, eval_every_steps=50, num_games=10):
        self.eval_every_steps = eval_every_steps
        self.num_games = num_games

    def on_step_end(self, args, state, control, model=None, processing_class=None, **kwargs):
        if state.global_step % self.eval_every_steps != 0:
            return

        tokenizer = processing_class
        if tokenizer is None or model is None:
            return

        was_training = model.training
        model.eval()

        wins = 0
        total_moves = 0
        invalid_count = 0
        for i in range(self.num_games):
            game = MinesweeperGame(rows=6, cols=6, num_mines=5, seed=10000 + i)
            moves = 0
            invalids = 0
            while game.state() == "ongoing" and moves < 50:
                prompt = format_state_for_llm(game)
                text = tokenizer.apply_chat_template(
                    [{"role": "user", "content": prompt}],
                    tokenize=False,
                    add_generation_prompt=True,
                    enable_thinking=False,   # Qwen3: non-thinking mode
                )
                with torch.no_grad():
                    output = model.generate(
                        **tokenizer(text, return_tensors="pt").to(model.device),
                        temperature=0.3,       # Low temp for eval (deterministic)
                        max_new_tokens=128,
                        do_sample=True,
                        top_p=0.8,
                        top_k=20,
                    )
                response = tokenizer.decode(output[0], skip_special_tokens=True)
                action = parse_llm_action(response)
                if action is None:
                    invalids += 1
                    if invalids >= 3:
                        break
                    continue
                invalids = 0
                game.do_action(action)
                moves += 1
            if game.state() == "success":
                wins += 1
            total_moves += moves
            invalid_count += invalids

        win_rate = wins / self.num_games
        avg_moves = total_moves / self.num_games
        print(f"\n[Eval @ step {state.global_step}] "
              f"Win: {wins}/{self.num_games} ({win_rate*100:.0f}%) | "
              f"Avg moves: {avg_moves:.1f} | "
              f"Invalid outputs: {invalid_count}\n")

        if was_training:
            model.train()

eval_callback = MinesweeperEvalCallback(eval_every_steps=50, num_games=10)
print("Eval callback: 10 games every 50 steps (temp=0.3 for deterministic eval)")

# Train the Model

Start GRPO training with reward functions:

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        valid_json_reward,   # Reward valid JSON format + conciseness
        gameplay_scores,     # Core gameplay (all 12 criteria)
        strategic_reward,    # Logical deduction bonuses
    ],
    args = training_args,
    train_dataset = dataset,
    callbacks = [eval_callback],  # Periodic gameplay evaluation
)

print("Starting GRPO training with 3 reward functions...")
print("  [1] valid_json_reward  (weight: 0.15)")
print("  [2] gameplay_scores    (weight: 0.70)")
print("  [3] strategic_reward   (weight: 0.15)")
trainer.train()

# Test Trained Model

Evaluate the finetuned model:

In [ ]:
# Test on new game
FastLanguageModel.for_inference(model)

test_game = MinesweeperGame(rows=6, cols=6, num_mines=5, seed=99)
test_prompt = format_state_for_llm(test_game)

test_text = tokenizer.apply_chat_template(
    [{"role": "user", "content": test_prompt}],
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False,   # Qwen3: non-thinking mode
)

print("=== Trained Model Response ===")
output = model.generate(
    **tokenizer(test_text, return_tensors = "pt").to("cuda"),
    temperature = 0.3,
    max_new_tokens = 128,
    do_sample = True,
    top_p = 0.8,
    top_k = 20,
    repetition_penalty = 1.2,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

# Parse and test action
response_text = tokenizer.decode(output[0], skip_special_tokens=True)
action = parse_llm_action(response_text)
print(f"\nParsed action: {action}")

if action:
    result = test_game.do_action(action)
    print(f"Action result: {result}")
    print(f"Game state: {test_game.state()}")
    print(test_game.pretty_print())
else:
    print("⚠️ Failed to parse a valid action from the response")

# Evaluation: Play Complete Games

Test the model on multiple complete games:

In [ ]:
def play_full_game(model, tokenizer, rows=6, cols=6, num_mines=5, seed=None,
                   max_moves=50, verbose=False):
    """Play a complete Minesweeper game with the model, tracking detailed metrics."""
    game = MinesweeperGame(rows=rows, cols=cols, num_mines=num_mines, seed=seed)
    moves = 0
    invalid_moves = 0
    logical_moves = 0
    flags_correct = 0
    flags_wrong = 0

    while game.state() == "ongoing" and moves < max_moves:
        prompt = format_state_for_llm(game)
        text = tokenizer.apply_chat_template(
            [{"role": "user", "content": prompt}],
            tokenize = False,
            add_generation_prompt = True,
            enable_thinking = False,   # Qwen3: non-thinking mode
        )

        with torch.no_grad():
            output = model.generate(
                **tokenizer(text, return_tensors="pt").to("cuda"),
                temperature = 0.3,       # Low temp for deterministic eval
                max_new_tokens = 128,
                do_sample = True,
                top_p = 0.8,
                top_k = 20,
                repetition_penalty = 1.2,
            )

        response = tokenizer.decode(output[0], skip_special_tokens=True)
        action = parse_llm_action(response)

        if action is None:
            invalid_moves += 1
            if invalid_moves >= 3:
                break
            continue

        invalid_moves = 0  # Reset streak on valid move

        # Track logical moves
        safe = _compute_safe_cells(game)
        mine = _compute_mine_cells(game)
        if action["type"] == "reveal" and [action["row"], action["col"]] in safe:
            logical_moves += 1
        elif action["type"] == "flag" and [action["row"], action["col"]] in mine:
            logical_moves += 1

        # Track flag accuracy
        if action["type"] == "flag":
            r, c = action["row"], action["col"]
            if 0 <= r < game.rows and 0 <= c < game.cols:
                if game._board[r][c] == -1:
                    flags_correct += 1
                else:
                    flags_wrong += 1

        if verbose:
            print(f"  Move {moves}: {action}")

        game.do_action(action)
        moves += 1

    return {
        "game": game,
        "moves": moves,
        "logical_moves": logical_moves,
        "flags_correct": flags_correct,
        "flags_wrong": flags_wrong,
        "result": game.state(),
    }


# ── Comprehensive Evaluation ──
NUM_EVAL_GAMES = 100
print(f"Evaluating model on {NUM_EVAL_GAMES} games...\n")

FastLanguageModel.for_inference(model)  # Enable Unsloth fast inference

wins = 0
total_moves = 0
total_logical = 0
total_flags_correct = 0
total_flags_wrong = 0
results_counter = {"success": 0, "failed": 0, "ongoing": 0}

for i in range(NUM_EVAL_GAMES):
    info = play_full_game(model, tokenizer, seed=i + 5000)
    result = info["result"]
    results_counter[result] = results_counter.get(result, 0) + 1

    if result == "success":
        wins += 1
    if i < 10 or result == "success":
        cells_revealed = len(info["game"]._revealed)
        total_safe = info["game"].rows * info["game"].cols - info["game"].num_mines
        tag = "🏆 WIN" if result == "success" else "💀 LOSS" if result == "failed" else "⏱ TIMEOUT"
        print(f"Game {i+1:3d}: {tag} | {info['moves']:2d} moves | "
              f"{info['logical_moves']} logical | "
              f"{cells_revealed}/{total_safe} revealed | "
              f"flags ✓{info['flags_correct']} ✗{info['flags_wrong']}")

    total_moves += info["moves"]
    total_logical += info["logical_moves"]
    total_flags_correct += info["flags_correct"]
    total_flags_wrong += info["flags_wrong"]

if NUM_EVAL_GAMES > 10:
    print(f"... (first 10 + wins shown; {NUM_EVAL_GAMES} total)")

print(f"\n{'='*55}")
print(f"  RESULTS ({NUM_EVAL_GAMES} games)")
print(f"{'='*55}")
print(f"  Win rate:            {wins}/{NUM_EVAL_GAMES} ({wins/NUM_EVAL_GAMES*100:.1f}%)")
print(f"  Avg moves/game:      {total_moves/NUM_EVAL_GAMES:.1f}")
print(f"  Avg logical moves:   {total_logical/NUM_EVAL_GAMES:.1f}")
print(f"  Flag accuracy:       {total_flags_correct}/{total_flags_correct+total_flags_wrong} "
      f"({total_flags_correct/(total_flags_correct+total_flags_wrong)*100:.1f}%)" if total_flags_correct + total_flags_wrong > 0 else "  Flags: none placed")
print(f"  Outcomes:            {results_counter}")
print(f"{'='*55}")

# Save the Model

Save your trained model for competition submission:

In [ ]:
# Save LoRA adapters
model.save_pretrained("my_minesweeper_model")
tokenizer.save_pretrained("my_minesweeper_model")
print("✅ LoRA adapters saved to: my_minesweeper_model/")

# Save merged model in 16bit (local file name which will be used for eval)
if True:
    model.save_pretrained_merged(
        "my_minesweeper_model_merged",
        tokenizer,
        save_method = "merged_16bit"
    )
    print("✅ Merged 16-bit model saved to: my_minesweeper_model_merged/")

# Improvements Applied (Research-Backed)

| # | Change | Source |
|---|--------|--------|
| 1 | Model: Qwen3-4B (thinking mode DISABLED) | Only model in cache; non-thinking = pure JSON |
| 2 | `chat_template_kwargs={"enable_thinking": False}` | Qwen3 docs + TRL GRPOConfig |
| 3 | LoRA rank 16→32, alpha=rank, dropout=0.05 | LoRA best practices |
| 4 | Prompt with pre-computed logical hints | Chain-of-thought (Wei 2022) |
| 5 | Complete 12-criterion `gameplay_scores` | Competition scoring rubric |
| 6 | `strategic_reward` — bonus for deducible moves | Reward shaping (Ng 1999) |
| 7 | `valid_json_reward` — conciseness + format | InstructGPT (2022) |
| 8 | Curriculum dataset: fresh→early→mid→late | Bengio et al. (2009) |
| 9 | 2000 training samples (up from 1000) | More diversity |
| 10 | `num_generations=8` (up from 4) | Better reward estimation |
| 11 | Cosine LR schedule + LR 2e-5 | Loshchilov & Hutter (2017) |
| 12 | `loss_type="dapo"` (default) | DAPO paper — eliminates length bias |
| 13 | `reward_weights=[0.15, 0.70, 0.15]` | TRL reward weighting |
| 14 | Qwen3 sampling: top_p=0.95, top_k=20 | Qwen3 best practices |
| 15 | Low temperature (0.3) at evaluation | Deterministic eval |
| 16 | `FastLanguageModel.for_inference()` | Unsloth fast inference |
| 17 | `max_seq_length=4096` (Qwen3 native 32K) | Qwen3 architecture |

## Further Tuning Ideas
- Increase `max_steps` to 1000+ for longer training
- Try `loss_type="dr_grpo"` (Dr. GRPO paper) to further reduce bias
- Set `scale_rewards="batch"` (PPO Lite paper) for batch-level normalization
- Add `mask_truncated_completions=True` for training stability (DAPO)
- Try `num_iterations=2` for generation reuse (speeds up training)
- Try enabling thinking mode (`enable_thinking=True`) with longer `max_completion_length=512` for chain-of-thought reasoning